# Stability Function

The Irea of this notebook is to investigate teh stability function of a Runge Kutta method with variable Weights.

The stbility function is 

$\displaystyle\sum_{i=0}^s R_i(z) b_i$ 

where $R_i(z)$ is the stability function for the Runge Kutta method with 
$ b_n = \begin{cases}
    1      & \quad \text{if } n=i\\
    0  & \quad \text{if } n \neq i
  \end{cases} $
  
 Because the stability functions are polynomials the stability functions form a $s$-dimentional vector space

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import sympy as sp

import matplotlib.colors as mcolors

#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23')
SDIRK34 = rk.loadRKM('SDIRK34')
SDIRK54 = rk.loadRKM('SDIRK54')
TR_BDF2 = rk.loadRKM('TR-BDF2')


#Extrapolation method
ex2 = rk.extrap(2,'implicit euler')
ex3 = rk.extrap(3,'implicit euler')
ex6 = rk.extrap(6,'implicit euler')
ex8 = rk.extrap(8,'implicit euler')




In [ ]:
rkm = ex3

s = len(rkm.b)


stab_functions= ['?'] * s
b_orig = rkm.b

for i in range(s):
    b= [0]*s
    b[i] = 1
    
    rkm.b = b
    stab_functions[i] = rkm.stability_function()
    
rkm.b = b_orig



In [ ]:
def plot_amp_stab(stab_functions,n):
    p,q = stab_functions[n]

    # Convert coefficients to floats for speed
    if p.coeffs.dtype=='object':
        p = np.poly1d([float(c) for c in p.coeffs])
    if q.coeffs.dtype=='object':
        q = np.poly1d([float(c) for c in q.coeffs])
        
    u = np.linspace(-10,10,200)
    v = np.linspace(-10,10,200)
    
    U,V = np.meshgrid(u,v)
    Q = U+V*1j
    R=np.abs(p(Q)/q(Q))

    plt.pcolormesh(U, V, np.log(R),cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
    plt.colorbar()
    plt.contour(U,V,R,[0,1],colors='k',alpha=1,linewidths=3)
    plt.grid()
    
    
plot_amp_stab(stab_functions,0)

In [ ]:
plot_amp_stab(stab_functions,1)

In [ ]:
plot_amp_stab(stab_functions,2)

In [ ]:
plot_amp_stab(stab_functions,3)

In [ ]:
plot_amp_stab(stab_functions,4)

In [ ]:
plot_amp_stab(stab_functions,5)

In [ ]:
print(rkm)

For $b = (0,0,0,1,0,0)$ the stability region is a circle inside the left halfeplane. Now wo test if this is realy the case or if it is due to errors in the plotting function. 
This realy ist the case.

In [ ]:
stab_functions[3]

In [ ]:
v = np.zeros_like(b_orig)
v[3] =1

rkm.b = v
rkm.plot_stability_region()

In [ ]:
print(rkm)

The method can be writen in a more compact way. Yhis is also interesting for the case with $b = (0,1,0,0,0,0)^T$ where the stability region is the left halfeplane. Apparently for an Runge Kutta method with

\begin{array}
{c|cc}
c_1 & a\\
\hline
& 1 
\end{array}

where $c_1 = 4$ the left halve plane is in the stability region for $a \geq 1/2$ (Not 100% shure for the $=$ case)

In [ ]:
kr13 = rk.RungeKuttaMethod(np.array([[1/3]]),np.array([1]))
print(kr13)

In [ ]:
kr13.plot_stability_region()

# Testing for stability

As seen for the case $b = (0,0,0,1,0,0)$ the resulting RK-Methods are sometimes not A-stable.
Maybee it is possible to derive a condition for the b that enshures that the new method is A-Stable.

This therefor the following two conditoons have to be fullfilled: (Hairer: Solving ODE II)

$a)$ $|R(iy)|\leq1 \forall y \in R$

$b)$ $R(z)$ is analytic for $ Re(z) \leq 0$

Condition $a)$ can be tested with the E-Polynomial. To setup the E-Polynomial for an arbritrary b the sum

$$R_{sum}(iy) \sum_{i=1}^s b_i R_i(iy) $$ 
has to be writen as 
$$R_{sum}(iy) = \frac{P_{sum}(iy)}{Q_{sum}(iy)}$$

This would be easy if $Q_1 = Q_2 = \cdots = Q_{sum}$. This is true because according to the Definition 

$$R(Z) = \frac{det(I-zA+z\mathbb{1}b^T)}{det(I-zA)} $$

$Q(z)$ does only depend on $A$ and $A$ is the same for all summed Methods. The Poles can be determined by the Eigenvalues of $A$ using

$$det(I-zA) = \frac{1}{z}^s det(\frac{1}{z} I-A) = \lambda^s  det(-\lambda I+A)$$

with $$ \lambda = -\frac{1}{z}$$

Because the Matrix $A$ is triangular for Diagonally Implicit Methods the EWs are determined by the entries on the diagonal. If these are greater than zero the poles are in the right halfplane

Important: $\sum_{i=1}^s b_i = 1$ has to be fullfilled if this approch on computing the stability funtion is being used

In [ ]:




def generate_stab_functions(rkm):
    """
    This function takes a rkm method an generates the stability function for 
    b = [1,0 ... ,0] up until b=[0,0 ... ,1]
    
    """
    s = len(rkm.b)

    stab_functions= ['?'] * s
    b_orig = rkm.b

    for i in range(s):
        b= [0]*s
        b[i] = 1
    
        rkm.b = b
        stab_functions[i] = rkm.stability_function()
    
    rkm.b = b_orig
    return stab_functions


In [ ]:

def calcE(stab_functions):
    """
    This function Takes a set of stability functions and generates the E-function 
    (Hairer(1996): Solving ODE II (Chapter IV))
    The E-function can be parameterized with different b's
    
    
    Parameters:
    stab_functions: set of stability functions generated by generate_stab_functions(rkm)
    
    returns: 
    b: a tuple of sympy symbols with (b_1,...,b_s)
    y: a sympy symbol for E(y)
    E: the general E function
    
    example:
    
    stab_functions = generate_stab_functions(ex3)
    (b0,b1,b2,b3,b4,b5),y,E = calcE(stab_functions)
    sp.collect(E.expand(),y)
    
    """
    #generate the b,s
    variables = ''
    for i in range(len(stab_functions)):
        variables = variables + 'b'+ str(i) + ' '
    
    b = eval("sp.symbols('"+ variables + "')" )
    
    x = sp.symbols('x')
    y = sp.symbols('y',real = True)

    exprp = 'b[0]*stab_functions[0][0](x)'
    for i in range(1,len(stab_functions)):
        exprp = exprp+'+b['+str(i)+']*stab_functions['+str(i)+'][0](x)'

    print(exprp)
    P = eval(exprp)

    Q = stab_functions[0][1](x)

    E = Q.subs(x,sp.I*y)*Q.subs(x,-sp.I*y)-P.subs(x,sp.I*y)*P.subs(x,-sp.I*y)
    E.simplify()

    return b,y,E
#(b0,b1,b2,b3,b4,b5) = b

In [ ]:
#Run some tests:
stab_functions = generate_stab_functions(ex3)

(b0,b1,b2,b3,b4,b5),y,E = calcE(stab_functions)

In [ ]:
sp.collect(E.expand(),y)

In [ ]:
#at y=o for arbritary b, should give 0 because R(0)= 1
sp.collect(E.expand(),y).subs(y,0).subs(b0,1-(b1+b2+b3+b4+b5)).simplify()

In [ ]:
# with sum(b) = 1
sp.collect(E.subs(b5,1-(b0+b1+b2+b3+b4)).simplify().expand(),y)


In [ ]:
# Case where it should be 0 everywhere
sp.collect(E.subs({b0:0,b1:1,b2:0,b3:0,b4:0,b5:0}).simplify().expand(),y)

When using the EX2 method we have theree variables to choose ($b_0,b_1,b_2$). Because $b_0+b_1+b_2 = 1$ we can set $b_2$ to $b_2=1-(b_0+b_1)$. We then have two parameters left over. These can be drawn on a 2D-graph.

The stability function is always $1$ at $z=0$. This corresponds to $E(0)=0$. For $y \approx 0$ the lowest power of $y$ controlls the behavior. For stability we need $a_2 \geq 0$. 

The behavior for $y \to \infty$ is controlled by the coefficent of the highest power of $y$ in this case $a_6$. 

For positifity in the intermediate regime we need the coefficent $a_4$. If $a_4 \geq 0$ then $E(iy) \geq 0$

We plot the cureves for $a_2(b_0,b_1) = 0$,$a_4(b_0,b_1) = 0$ and $a_6(b_0,b_1) = 0$. 

The condition $a_2 \geq 0 ^ a_4 \geq 0 ^ a_6 \geq 0$ is sufficient but not necessary for $E(iy) \geq 0 \forall y \in R$
We can derive a sufficient and necessary condition from the zeros of $E(iy)$:
We know that $E(iy)$ is a polynomial of order 6 and can therefore only have 6 zeros. We know there is at least one zero at $y=0$, in fact, by symetry we know there is a double zero at $y=0$. 
There are two other potential zeros for $y \geq 0$. We can derive the positions of these zeros using the Quadratic formula. For this we substitute $y^2$ by $x$, divive by $x$ and get $a_2 + a_4 x + a_6 x^2 = 0$.
Using the discriminant we get a conditon for the existenc of the zeros for $x$ with $\sqrt{a_4^2 - 4a_2a_6} \geq 0$ we plot the boundary $\sqrt{a_4^2 - 4a_2a_6} = 0$.
When interpreting the results we have to take into account that zeros for $x<0$ do not lead to zeros for $y \in R$. 
The x are positive if $a_4 < \sqrt{a_4^2 - 4a_2a_6}$. Only the points with $4a_2\geq 0 $and $ a_6 \geq 0$ are relevant so we can rewrite the statement as $a_4 < \sqrt{a_4^2} = |a_4| \Leftrightarrow a_4 < 0$ 
This means in effect that the region for $\sqrt{a_4^2 - 4a_2a_6} \leq 0$ and $a_4(b_0,b_1) \geq 0$ have to be combined.

In [ ]:
stab_functions = generate_stab_functions(ex2)
(b0,b1,b2),y,E = calcE(stab_functions)
E_collected = sp.collect(E.subs(b2,1-(b0+b1)).expand(),y)

In [ ]:


coeffs = []
for i in range(0,4*2): #until the maximum stage*2
    print('y^',i,':')
    coefficent = E_collected.coeff(y, i)
    print(coefficent)
    coeffs.append(coefficent)

    
back = (b0,b1) #storing the sympy variables because i am goint o overwrite these to evaluate the expression on a grid 
    
size = 5 #Size of ploted area
 
power = [2,4,6]
colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728']  
labels = [r'$a_2 = 0$',r'$a_4 = 0$',r'$a_6 = 0$',r'$\sqrt{a_4^2 - 4a_2a_6} = 0$']

w =['?'] * 4

plt.figure()
(b0,b1) = np.meshgrid(np.linspace(-size,size,100),np.linspace(-size,size,100))
for i in range(3):
    w[i] = eval(str(coeffs[power[i]]))
    plt.contour(b0,b1,w[i],[0],colors=colors[i],alpha=1,linewidths=3)

plt.grid()


#exact condition from formula for zeros:

dis = coeffs[4]**2-4*coeffs[6]*coeffs[2]
print('discriminant:',dis)
w[3] = eval(str(dis))
plt.contour(b0,b1,w[3],[0],colors='#d62728',alpha=1,linewidths=3)


#expression for region with A-stability
A = 1.*((w[0]>=0) & (w[2]>=0) & ((w[1]>=0) | (w[3]<0)))

plt.pcolormesh(b0,b1,A,cmap=plt.get_cmap('Greys'),vmin=0, vmax=4)

(b0,b1) = back   

plt.figure()
for i in range(4):
    plt.plot([1,1],[1,1],color=colors[i],label=labels[i])
    
plt.plot([1,1],[1,1],color='#AFAFAF',linewidth = 10,label='Region with A-Stability')
plt.legend()

In [ ]:

plt.pcolor(w[3],cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
plt.colorbar()

In [ ]:
#plotting E(iy) for a set of bs to enshure if the plot is correct

b0_num = 4
b1_num = -1.915
b2_num = 1- (b0_num+b1_num)



E_expr = str(E.subs({b0:b0_num,b1:b1_num,b2:b2_num}).expand())

p = sp.Poly(E_expr)
print(str(p))
p = np.poly1d(p.all_coeffs())

print(p)


y_num = np.linspace(-10,10,1000)
plt.plot(y_num,p(y_num))
plt.grid()
plt.ylim([-1,1])


Test functions